In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE


# Load the dataset
data = pd.read_csv('/content/German Credit Data.txt', header=None)  # Update filename as needed

# Check the shape of the dataset
print(f"Dataset shape: {data.shape}")

# Separate features (X) and labels (y)
X = data.iloc[:, :-1]
y = data.iloc[:, -1]  # Class labels are in the last column


Dataset shape: (1000, 21)


In [2]:
# Load the file
input_file = '/content/German Credit Data.txt'  # Replace with your file path
output_file = '/content/German Credit Data1.txt'  # Specify the output file name

# Drop the 20th column (index 19 since Python uses 0-based indexing)
data = data.drop(data.columns[19], axis=1)

# Save the modified file
data.to_csv(output_file, index=False)

In [3]:
# Define the column names as a list
column_names = [
    'Status', 'Duration', 'Credit_history', 'Purpose',
    'Credit_amount', 'Savings_account', 'Present_employment', 'Installment_rate',
    'sex', 'guarantors', 'residence', 'Property', 'Age', 'installment', 'Housing', 'existing_credits',
    'job', 'num_dependents', 'telephone', 'foreign_worker'
]

# Assign column names to the dataset
data.columns = column_names

# Verify the column names
print(data.head())


  Status  Duration Credit_history Purpose  Credit_amount Savings_account  \
0    A11         6            A34     A43           1169             A65   
1    A12        48            A32     A43           5951             A61   
2    A14        12            A34     A46           2096             A61   
3    A11        42            A32     A42           7882             A61   
4    A11        24            A33     A40           4870             A61   

  Present_employment  Installment_rate  sex guarantors  residence Property  \
0                A75                 4  A93       A101          4     A121   
1                A73                 2  A92       A101          2     A121   
2                A74                 2  A93       A101          3     A121   
3                A74                 2  A93       A103          4     A122   
4                A73                 3  A93       A101          4     A124   

   Age installment Housing  existing_credits   job  num_dependents telepho

In [4]:
print(data.columns)

Index(['Status', 'Duration', 'Credit_history', 'Purpose', 'Credit_amount',
       'Savings_account', 'Present_employment', 'Installment_rate', 'sex',
       'guarantors', 'residence', 'Property', 'Age', 'installment', 'Housing',
       'existing_credits', 'job', 'num_dependents', 'telephone',
       'foreign_worker'],
      dtype='object')


In [5]:
# Rename the last column to 'label' if not already
data.rename(columns={data.columns[-1]: 'label'}, inplace=True)

# Split into features and labels
X = data.drop(columns=['label'])
y = data['label']

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Encode categorical features
categorical_columns = X.select_dtypes(include=['object']).columns  # Identify categorical columns

# Apply LabelEncoder to each categorical column
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Store encoders for inverse_transform if needed

In [7]:
# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [8]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# For the original data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# For the balanced data
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(
    X_smote, y_smote, test_size=0.3, random_state=42)



Train a Machine Learning Model

In [9]:
# Initialize Random Forest Classifier
model = RandomForestClassifier(random_state=42, class_weight='balanced')

# Train on imbalanced dataset
model.fit(X_train, y_train)

# Train on balanced dataset (e.g., SMOTE)
model_smote = RandomForestClassifier(random_state=42)
model_smote.fit(X_train_balanced, y_train_balanced)

RandomForestClassifier(random_state=42)

In [10]:
# Predict on test data
y_pred = model.predict(X_test)
y_pred_smote = model_smote.predict(X_test_balanced)

# Calculate F-score
f1_original = f1_score(y_test, y_pred, average='weighted')
f1_smote = f1_score(y_test_balanced, y_pred_smote, average='weighted')

# Print Classification Reports
print("Classification Report for Imbalanced Data:")
print(classification_report(y_test, y_pred))

print("Classification Report for Balanced Data (SMOTE):")
print(classification_report(y_test_balanced, y_pred_smote))

# Output F-scores
print(f"F1-Score on Original Data: {f1_original}")
print(f"F1-Score on SMOTE Data: {f1_smote}")


Classification Report for Imbalanced Data:
              precision    recall  f1-score   support

           1       0.77      0.90      0.83       209
           2       0.63      0.37      0.47        91

    accuracy                           0.74       300
   macro avg       0.70      0.64      0.65       300
weighted avg       0.73      0.74      0.72       300

Classification Report for Balanced Data (SMOTE):
              precision    recall  f1-score   support

           1       0.85      0.78      0.82       218
           2       0.79      0.85      0.82       202

    accuracy                           0.82       420
   macro avg       0.82      0.82      0.82       420
weighted avg       0.82      0.82      0.82       420

F1-Score on Original Data: 0.7210221043324492
F1-Score on SMOTE Data: 0.8166489983881239
